In [ ]:
!pip install turicreate

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import turicreate as tc
import numpy as np # linear algebra
import pandas as pd 
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.externals import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
product_data = tc.SFrame('/content/gdrive/My Drive/Product Recommender System/all_csv_files.csv')

Finished parsing file /content/gdrive/My Drive/Product Recommender System/all_csv_files.csv

Parsing completed. Parsed 100 lines in 3.82425 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /content/gdrive/My Drive/Product Recommender System/all_csv_files.csv

Parsing completed. Parsed 1048575 lines in 1.32908 secs.

In [ ]:
product_data

user_id,product_id,rating,time_stamp
B013CG1EVC,A3GDQGSD4C0HOB,5,1496966400
B013CG1EVC,A1C88HIIEVN0QN,5,1445904000
B013CG1EVC,A1G5MTDE3A5LSL,5,1440892800
B013CG1XS6,A3SOHSDN51M5NQ,5,1441497600
B0096E8EPY,AYRFEPLLS7A7O,3,1368403200
B0096E8EPY,A7WT61P4D6W0H,1,1368403200
B0096E8EPY,A2KH4WTPYWYQST,4,1368403200
B0096E8EPY,A204ETWOV23HO4,3,1368403200
B0096E8EPY,A1XT4N33B6MOP9,5,1368403200
B0096E8EPY,A1JEJEDR7NDZ7C,3,1368403200


In [ ]:
product_data['product_id'].show()

Materializing SArray

In [ ]:
tc.visualization.histogram(product_data['rating'], xlabel='Rating', ylabel='Total Number of Rating', title='Distributaion of Rating')

Materializing SArray

In [ ]:
len(product_data)

1048575

In [ ]:
X,Y = product_data.random_split(0.7,seed=0)
train_data,test_data =X.random_split(0.7,seed=0)


In [ ]:
train_data
users = train_data['user_id'].unique()
type(users)


user_id,product_id,rating,time_stamp
B013CG1EVC,A3GDQGSD4C0HOB,5,1496966400
B013CG1EVC,A1C88HIIEVN0QN,5,1445904000
B0096E8EPY,AYRFEPLLS7A7O,3,1368403200
B0096E8EPY,A7WT61P4D6W0H,1,1368403200
B0096E8EPY,A2KH4WTPYWYQST,4,1368403200
B0096E8EPY,A204ETWOV23HO4,3,1368403200
B0096E8EPY,A1XT4N33B6MOP9,5,1368403200
B0096E8EPY,A1JEJEDR7NDZ7C,3,1368403200
B0096E8EPY,A3LX65FO3T9Z8N,4,1368403200
B0096E8EPY,AEBFJCJ8VJ2ER,1,1530316800


turicreate.data_structures.sarray.SArray

#1)Popularity Based Model

In [ ]:
popularity_model = tc.popularity_recommender.create(train_data,
                                                    user_id = 'user_id',
                                                    item_id = 'product_id')

Warning: Ignoring columns rating, time_stamp;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 514015 observations with 41455 users and 475112 items.

Data prepared in: 0.967613s

514015 observations to process; with 475112 unique items.

##Predictions using Popularity Based Model

In [ ]:
popularity_model.recommend(users=[users[0]])

user_id,product_id,score,rank
B00QAB4KWC,A2E22B6U6A4NQK,36.0,1
B00QAB4KWC,A22CW0ZHY3NJH8,22.0,2
B00QAB4KWC,A1O35FYKDTF5Z9,18.0,3
B00QAB4KWC,A1UQBFCERIP7VJ,17.0,4
B00QAB4KWC,A7Y6AVS576M03,14.0,5
B00QAB4KWC,A1KSC91G9AIY2Z,12.0,6
B00QAB4KWC,A3QG0HA91HZWXK,12.0,7
B00QAB4KWC,A2GJX2KCUSR0EI,11.0,8
B00QAB4KWC,A3VZXF1X0XG9AU,11.0,9
B00QAB4KWC,AAT47MLSLM5CQ,11.0,10


##2) Item Similarity Model

In [ ]:
personalised_model = tc.item_similarity_recommender.create(train_data,
                                                           user_id = 'user_id',
                                                           item_id = 'product_id')

Warning: Ignoring columns rating, time_stamp;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 514015 observations with 41455 users and 475112 items.

...

| 41.99s                              | 52.25            | 249336          |

| 45.43s                              | 59.25            | 281756          |

| 47.16s                              | 63.25            | 301005          |

| 50.47s                              | 66.75            | 317297          |

| 57.01s                              | 76.25            | 363031          |

In [ ]:
personalised_model.recommend(users=[users[1]])

In [ ]:
personalised_model.get_similar_items(['A2D75NBPXJMBVJ'])


In [ ]:
personalised_model.recommend(users=[users[7]])

##3) Item-Item Collaborative Filtering


In [ ]:
!pip install surprise

In [ ]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
from surprise.model_selection import train_test_split



In [ ]:
import pandas as pd

url = 'https://drive.google.com/file/d/1n-1vM3QApuMDw0y_ucxHng16CrNiMmIi/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path)

In [ ]:
df.head()

In [ ]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
from surprise.model_selection import train_test_split

In [ ]:
df.drop(['time_stamp'], axis=1, inplace= True)
df = df[:-1030000]


df

In [ ]:
reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df,reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=0.3,random_state=10)

##Computing the pearson_baseline similarity between all pairs of users (or items).

In [ ]:
algo = KNNWithMeans(k=5, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

##Running the test model against the trained model.

In [ ]:
test_prediction = algo.test(testset)

In [ ]:
test_prediction

##Calculating RMSE

In [ ]:
accuracy.rmse(test_prediction, verbose=True)

##Model Based Approach.

In [ ]:
ratings_matrix = df.pivot_table(values='rating', index='user_id', columns='product_id', fill_value=0)
ratings_matrix.head()

In [ ]:
ratings_matrix.shape

In [ ]:
X = ratings_matrix.T
X.head()

In [ ]:
X.shape

In [ ]:
import numpy as np
correlation_matrix = np.corrcoef(X)
correlation_matrix.shape

In [ ]:
X.index[5000]

###The product A23GQAUN3MJGNB is purchased by a customer.

In [ ]:
i = "A23GQAUN3MJGNB"

product_names = list(X.index)
product_ID = product_names.index(i)
product_ID

###Correlation for all items with the item purchased by this customer based on items rated by other customers people who bought the same product

In [ ]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

##Recommending top 30 highly correlated products.

In [ ]:
Recommend = list(X.index[correlation_product_ID > 0.85])

# Removing the item already bought by the customer
Recommend.remove(i) 

Recommend[0:30]

NameError: ignored

#Model Performance Comparison using Precision-Recall.

In [ ]:
%matplotlib inline
model_performance = tc.recommender.util.compare_models(test_data,
                                                       [popularity_model, personalised_model],
                                                       user_sample = 1)

In [ ]:
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Average Over Users")

plt.plot(model_performance[1]['precision_recall_overall']['precision'],
         model_performance[1]['precision_recall_overall']['recall'],
          label = ["personalised_model"])
plt.plot (model_performance[0]['precision_recall_overall']['precision'],
         model_performance[0]['precision_recall_overall']['recall'],
         label = ["popularity_model"])

In [ ]:
popularity_model.evaluate(test_data, metric= 'precision_recall')

##K-Fold Cross validation of CF Model (pearson_baseline similarity).

In [ ]:
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import KFold


kf = KFold(n_splits=5)

algo = SVD()

for trainset, testset in kf.split(data):

    
    algo.fit(trainset)
    predictions = algo.test(testset)

    
    accuracy.rmse(predictions, verbose=True)

##Precision-Recall curve for CF model

###Computing Average Precission Recall Score

##Computing the cosine similarity between all pairs of users (or items).

In [ ]:
algo2 = KNNWithMeans(k=5, sim_options={'name': 'cosine', 'user_based': False})
algo2.fit(trainset)

In [ ]:

test_prediction2 = algo2.test(testset)
test_prediction2


In [ ]:
accuracy.rmse(test_prediction2, verbose=True)



In [ ]:
ratings_matrix2 = df.pivot_table(values='rating', index='user_id', columns='product_id', fill_value=0)
ratings_matrix2.head()

In [ ]:
ratings_matrix2.shape


In [ ]:
X2 = ratings_matrix2.T
X2.head()


In [ ]:
correlation_matrix2 = np.corrcoef(X2)
correlation_matrix2.shape

In [ ]:
X2.index[5000]
i2 = "A23GQAUN3MJGNB"

product_names2 = list(X2.index)


In [ ]:
product_ID2 = product_names2.index(i2)
product_ID2

In [ ]:
correlation_product_ID2 = correlation_matrix[product_ID2]
correlation_product_ID2.shape


In [ ]:
Recommend2 = list(X2.index[correlation_product_ID2 > 0.85])

# Removing the item already bought by the customer
Recommend2.remove(i2) 

Recommend2[0:30]

In [ ]:
algo2 = SVD()

for trainset, testset in kf.split(data):

    
    algo2.fit(trainset)
    predictions2 = algo2.test(testset)

    
    accuracy.rmse(predictions2, verbose=True)